In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vasil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vasil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import math
from itertools import product, count
from string import punctuation
from heapq import nlargest

In [3]:
stopwords = set(stopwords.words('english') + list(punctuation))

### Calculate Similarity ###
Calculate the similarity between two sentences

TextRank Algorithm

$$Similarity\left( S _ { i } , S _ { j } \right) = \frac { \left| \left\{ w _ { k } | w _ { k } \in S _ { i } \& w _ { k } \in S _ { j } \right\} \right| } { \log \left( \left| S _ { i } \right| \right) + \log \left( \left| S _ { j } \right| \right) }$$

$S _ { i }$ represents the ith sentence <br>
$w _ { k }$ represents the kth word in a sentence <br>
$|S _ { i }|$ represents the number of words in a sentence <br>
${ \left\{ w _ { k } | w _ { k } \in S _ { i } \& w _ { k } \in S _ { j } \right\} }$ represents the number of words appear in both $S _ { i }$ and $S _ { j }$

In [4]:
def calculate_similarity(sen1, sen2):
    counter = 0
    for word in sen1:
        if word in sen2:
            counter += 1
    return counter / (math.log(len(sen1)) + math.log(len(sen2)))

### Create Similarity Adjacency Matrix ###

In [5]:
"""
传入句子的列表
返回各个句子之间相似度的图
（邻接矩阵表示）
"""
def create_graph(word_sent):
    num = len(word_sent)
    # 初始化表
    board = [[0.0 for _ in range(num)] for _ in range(num)]

    for i, j in product(range(num), repeat=2):
        if i != j:
            board[i][j] = calculate_similarity(word_sent[i], word_sent[j])
    return board

Based on the similarity algorithm, we can calculate the frequency score with TextRank

$$W S \left( V _ { i } \right) = ( 1 - d ) + d * \sum _ { V _ { j } \in \operatorname { In } \left( V _ { i } \right) } \frac { w _ { j i } } { \sum _ { V _ { k } \in O u t \left( V _ { j } \right) } w _ { j k } } W S \left( V _ { j } \right)$$

$W S \left( V _ { i } \right)$ represents the score of sentence $V _ { i }$ <br>
$d$ is the Damping Constant,  $(0<d<1)$

In [6]:
"""
输入相似度邻接矩阵
返回各个句子的分数
"""
def weighted_pagerank(weight_graph):
    # 把初始的分数值设置为0.5
    scores = [0.5 for _ in range(len(weight_graph))]
    old_scores = [0.0 for _ in range(len(weight_graph))]

    # 开始迭代
    while different(scores, old_scores):
        for i in range(len(weight_graph)):
            old_scores[i] = scores[i]

        for i in range(len(weight_graph)):
            scores[i] = calculate_score(weight_graph, scores, i)
    return scores

"""
判断前后分数有没有变化
前后差距小于0.0001
分数就趋于稳定
"""
def different(scores, old_scores):
    flag = False
    for i in range(len(scores)):
        if math.fabs(scores[i] - old_scores[i]) >= 0.0001:
            flag = True
            break
    return flag

"""
根据公式求出指定句子的分数
"""
def calculate_score(weight_graph, scores, i):
    length = len(weight_graph)
    d = 0.85
    added_score = 0.0

    for j in range(length):
        fraction = 0.0
        denominator = 0.0
        # 计算分子
        fraction = weight_graph[j][i] * scores[j]
        # 计算分母
        for k in range(length):
            denominator += weight_graph[j][k]
        added_score += fraction / denominator
    # final score
    weighted_score = (1 - d) + d * added_score

    return weighted_score

### Find n sentences with highest scores ###

In [7]:
def Summarize(text,n):
    # tokenize sentences
    sents = sent_tokenize(text)
    # tokenize words
    # word_sent is a 2D list
    # word_sent[i] represents the ith sentence
    # word_sent[i][j] represents the jth word in the ith sentence
    word_sent = [word_tokenize(s.lower()) for s in sents]

    # filter out stopwords
    for i in range(len(word_sent)):
        for word in word_sent[i]:
            if word in stopwords:
                word_sent[i].remove(word)
    similarity_graph = create_graph(word_sent)
    scores = weighted_pagerank(similarity_graph)
    sent_selected = nlargest(n, zip(scores, count()))
    sent_index = []
    for i in range(n):
        sent_index.append(sent_selected[i][1])
    return [sents[i] for i in sent_index]

### Run ###

In [8]:
if __name__ == '__main__':
    with open("news.txt", "r") as myfile:
        text = myfile.read().replace('\n' , '')
    print(Summarize(text, 2))

['The PHE website and app has a quiz that gives users a health score based on their lifestyle habits by asking questions such as, "Which snacks do you eat in a normal day?"', 'The campaign\'s clinical adviser, Prof Muir Gray, said it was about trying to make people have a different attitude to an "environmental problem".']
